In [1]:
import sys
sys.path.insert(0,'../scripts')
from pycap_for_PESTPP_MOU import prepare_MOU_files, postprocess_MOU, plot_pareto, prep_for_viz, create_viz_app
from pathlib import Path
from ipywidgets import interact, fixed
import yaml



# Read in one of the configuration files and postprocess it. 
### The `config_file` should be your `secnario_name` + `".yml"` as assigned in the `05_MOU_setup_run.ipynb` notebook. 

In [2]:
config_file = 'MOU_fish_cash_baseline.yml'

with open(Path('./configurations') / config_file,'r') as ifp:
        inpars = yaml.safe_load(ifp)
inpars

{'depletion_potential_threshold': 0.01,
 'objectives': 'fish_dollars',
 'pump_lbound_fraction': 0.69,
 'pump_ubound_fraction': 1.1,
 'ref_q': 8.6,
 'run_path': '../pycap_runs/pycap_pest/run_MOU_fish_cash_baseline',
 'scenario_name': 'MOU_fish_cash_baseline'}

### Below, the `config_file` is processed, beginning the postprocessing

In [3]:
run_path = Path(inpars['run_path'])
run_name = inpars['scenario_name']
pareto_df = postprocess_MOU(run_name, run_path)

### Next the evolution of the pareto frontier is plotted, over MOU iterations. Check out how the algorithm converges on a well-defined pareto tradeoff frontier

In [4]:
interact(plot_pareto,  currgen=(0,50,1), pareto_df=fixed(pareto_df));

interactive(children=(IntSlider(value=25, description='currgen', max=50), Output()), _dom_classes=('widget-int…

### Finally, there is an interactive plot with the pareto frontier on the left and a map of corresponding "optimal" pumping rates on the right. 

### Note - if you get an error about ports being in conflict, change the `port=4242` argument to a different number than `4242`

In [5]:
dollars=False
if 'dollar' in inpars['objectives']:
    dollars = True
pareto_df_final, dv_df =  prep_for_viz(pareto_df, 50, run_path, run_name, dollar_objective=dollars)
app = create_viz_app(pareto_df_final, dv_df)
app.run(port=4242)

/Users/mnfienen/miniforge3/envs/pycap/lib/python3.11/site-packages/pyogrio/__init__.py:7: DeprecationWarning: The 'shapely.geos' module is deprecated, and will be removed in a future version. All attributes of 'shapely.geos' are available directly from the top-level 'shapely' namespace (since shapely 2.0.0).


error parsing metadata from 'obsnme', continuing


KeyError: 'gen=42_member=5250_pso'

### check out `pareto_df_final` and `dv_df`. What's in these datasets? What further analysis can you perform with this information?

In [ ]:
dv_df.sample(10)

In [ ]:
pareto_df_final.sample(10)